In [2]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 1 : Installation et Imports
# ════════════════════════════════════════════════════════════════════

import boto3
import psycopg2
from psycopg2 import sql
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
from botocore.exceptions import ClientError

print("✅ Imports OK")
print("\n📦 Versions installées :")
print(f"   • boto3 : {boto3.__version__}")
print(f"   • psycopg2 : {psycopg2.__version__}")

✅ Imports OK

📦 Versions installées :
   • boto3 : 1.40.68
   • psycopg2 : 2.9.11 (dt dec pq3 ext lo64)


In [6]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Vérification du Fichier .env
# ════════════════════════════════════════════════════════════════════

print("\n⚙️ VÉRIFICATION DE LA CONFIGURATION\n")
print("="*60)

# Charger les variables d'environnement
load_dotenv()

# Variables requises
required_vars = {
    'AWS': ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_REGION', 'AWS_S3_BUCKET'],
    'RDS': ['DB_HOST', 'DB_PORT', 'DB_NAME', 'DB_USER', 'DB_PASSWORD']
}

# Vérifier chaque catégorie
all_ok = True

for category, vars_list in required_vars.items():
    print(f"\n🔍 {category} :")
    missing = []
    
    for var in vars_list:
        value = os.getenv(var)
        if value:
            # Masquer les valeurs sensibles
            if 'PASSWORD' in var or 'SECRET' in var:
                display_value = '*' * 8
            elif 'KEY' in var:
                display_value = value[:8] + '...' if len(value) > 8 else value
            else:
                display_value = value
            
            print(f"   ✅ {var:25s} : {display_value}")
        else:
            print(f"   ❌ {var:25s} : MANQUANT")
            missing.append(var)
            all_ok = False
    
    if missing:
        print(f"\n   ⚠️  Variables manquantes : {', '.join(missing)}")

if all_ok:
    print(f"\n{'='*60}")
    print("✅ Toutes les variables d'environnement sont configurées !")
    print("="*60)
else:
    print(f"\n{'='*60}")
    print("❌ Configuration incomplète - Vérifiez votre fichier .env")
    print("="*60)


⚙️ VÉRIFICATION DE LA CONFIGURATION


🔍 AWS :
   ✅ AWS_ACCESS_KEY_ID         : AKIASBCN...
   ✅ AWS_SECRET_ACCESS_KEY     : ********
   ✅ AWS_REGION                : eu-west-3
   ✅ AWS_S3_BUCKET             : 251107-140505-jedha-kayak-project

🔍 RDS :
   ✅ DB_HOST                   : kayak-db.cvumwuoaepex.eu-west-3.rds.amazonaws.com
   ✅ DB_PORT                   : 5432
   ✅ DB_NAME                   : kayak-db
   ✅ DB_USER                   : kayakdb
   ✅ DB_PASSWORD               : ********

✅ Toutes les variables d'environnement sont configurées !


In [8]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 3 : Test de Connexion S3
# ════════════════════════════════════════════════════════════════════

print("\n\n📦 TEST DE CONNEXION S3\n")
print("="*60)

try:
    # Initialiser le client S3
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name=os.getenv('AWS_REGION')
    )
    
    # Lister les buckets
    response = s3_client.list_buckets()
    
    print("✅ Connexion S3 réussie !")
    print(f"\n📦 Buckets S3 ({len(response['Buckets'])}) :")
    
    bucket_name = os.getenv('AWS_S3_BUCKET')
    bucket_exists = False
    
    for bucket in response['Buckets']:
        is_target = '✓' if bucket['Name'] == bucket_name else ' '
        print(f"   [{is_target}] {bucket['Name']}")
        if bucket['Name'] == bucket_name:
            bucket_exists = True
    
    # Vérifier le bucket cible
    if bucket_exists:
        print(f"\n✅ Bucket cible '{bucket_name}' trouvé")
        
        # Lister quelques fichiers
        try:
            response = s3_client.list_objects_v2(
                Bucket=bucket_name,
                MaxKeys=10
            )
            
            if 'Contents' in response:
                print(f"\n📁 Fichiers dans le bucket ({response['KeyCount']}) :")
                for obj in response['Contents'][:5]:
                    size_mb = obj['Size'] / (1024 * 1024)
                    print(f"   • {obj['Key']:50s} ({size_mb:.2f} MB)")
                
                if response['KeyCount'] > 5:
                    print(f"   ... et {response['KeyCount'] - 5} autres fichiers")
            else:
                print("\n📭 Bucket vide")
        
        except ClientError as e:
            print(f"\n⚠️  Impossible de lister les fichiers : {e}")
    
    else:
        print(f"\n⚠️  Bucket cible '{bucket_name}' NON trouvé")
        print("   Le bucket sera créé lors du déploiement")

except ClientError as e:
    print(f"❌ Erreur de connexion S3 : {e}")
    print("\n💡 Vérifications :")
    print("   1. AWS_ACCESS_KEY_ID est correct")
    print("   2. AWS_SECRET_ACCESS_KEY est correct")
    print("   3. Votre utilisateur IAM a les permissions S3")

except Exception as e:
    print(f"❌ Erreur inattendue : {e}")



📦 TEST DE CONNEXION S3

✅ Connexion S3 réussie !

📦 Buckets S3 (1) :
   [✓] 251107-140505-jedha-kayak-project

✅ Bucket cible '251107-140505-jedha-kayak-project' trouvé

📭 Bucket vide


In [9]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 4 : Test de Connexion RDS PostgreSQL
# ════════════════════════════════════════════════════════════════════

print("\n\n🗄️ TEST DE CONNEXION RDS PostgreSQL\n")
print("="*60)

# Configuration RDS
RDS_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT', 5432)),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# Afficher la configuration (masquer le mot de passe)
print("📋 Configuration RDS :")
for key, value in RDS_CONFIG.items():
    if key == 'password':
        print(f"   • {key:12s} : {'*' * 8}")
    else:
        print(f"   • {key:12s} : {value}")

print("\n🔗 Test de connexion...")

try:
    # Tenter la connexion
    conn = psycopg2.connect(**RDS_CONFIG)
    cursor = conn.cursor()
    
    print("✅ Connexion RDS réussie !\n")
    
    # Version PostgreSQL
    cursor.execute("SELECT version();")
    version = cursor.fetchone()[0]
    print(f"📊 PostgreSQL : {version.split(',')[0]}")
    
    # Taille de la base de données
    cursor.execute("""
        SELECT pg_size_pretty(pg_database_size(current_database())) as size;
    """)
    db_size = cursor.fetchone()[0]
    print(f"💾 Taille de la base : {db_size}")
    
    # Nombre de connexions actives
    cursor.execute("""
        SELECT count(*) FROM pg_stat_activity;
    """)
    connections = cursor.fetchone()[0]
    print(f"🔌 Connexions actives : {connections}")
    
    # Lister les tables existantes
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = cursor.fetchall()
    
    if tables:
        print(f"\n📊 Tables existantes ({len(tables)}) :")
        for table in tables:
            # Compter les lignes
            cursor.execute(f"SELECT COUNT(*) FROM {table[0]};")
            count = cursor.fetchone()[0]
            print(f"   • {table[0]:30s} ({count} lignes)")
    else:
        print("\n📭 Aucune table dans la base de données")
        print("   Les tables seront créées lors de la prochaine étape")
    
    cursor.close()
    conn.close()

except psycopg2.OperationalError as e:
    print(f"❌ Erreur de connexion : {e}\n")
    print("💡 Vérifications :")
    print("   1. L'instance RDS est démarrée")
    print("   2. Le endpoint (RDS_HOST) est correct")
    print("   3. Le groupe de sécurité autorise votre IP sur le port 5432")
    print("   4. Le mot de passe est correct")
    print("\n🔧 Pour autoriser votre IP :")
    print("   1. AWS Console → RDS → Votre instance")
    print("   2. Security Groups → Modifier les règles entrantes")
    print("   3. Ajouter : Type=PostgreSQL, Port=5432, Source=Mon IP")

except psycopg2.Error as e:
    print(f"❌ Erreur PostgreSQL : {e}")

except Exception as e:
    print(f"❌ Erreur inattendue : {e}")



🗄️ TEST DE CONNEXION RDS PostgreSQL

📋 Configuration RDS :
   • host         : kayak-db.cvumwuoaepex.eu-west-3.rds.amazonaws.com
   • port         : 5432
   • database     : kayak-db
   • user         : kayakdb
   • password     : ********

🔗 Test de connexion...
✅ Connexion RDS réussie !

📊 PostgreSQL : PostgreSQL 17.4 on aarch64-unknown-linux-gnu
💾 Taille de la base : 7795 kB
🔌 Connexions actives : 9

📭 Aucune table dans la base de données
   Les tables seront créées lors de la prochaine étape


In [10]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 5 : Récapitulatif de la Configuration
# ════════════════════════════════════════════════════════════════════

print("\n\n" + "="*60)
print("📋 RÉCAPITULATIF DE LA CONFIGURATION")
print("="*60)

# État des services
services_status = {
    '📦 S3': None,
    '🗄️ RDS PostgreSQL': None
}

# Test S3
try:
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name=os.getenv('AWS_REGION')
    )
    s3_client.list_buckets()
    services_status['📦 S3'] = '✅ Opérationnel'
except:
    services_status['📦 S3'] = '❌ Erreur'

# Test RDS
try:
    conn = psycopg2.connect(**RDS_CONFIG)
    conn.close()
    services_status['🗄️ RDS PostgreSQL'] = '✅ Opérationnel'
except:
    services_status['🗄️ RDS PostgreSQL'] = '❌ Erreur'

# Afficher
print("\n🔍 État des services AWS :\n")
for service, status in services_status.items():
    print(f"   {service:20s} : {status}")

# Recommandations
print("\n\n🎯 PROCHAINES ÉTAPES :\n")

if services_status['📦 S3'] == '✅ Opérationnel':
    print("   1. ✅ S3 configuré → Passez au notebook 06_deploy_s3.ipynb")
else:
    print("   1. ⚠️  Corrigez la configuration S3 avant de continuer")

if services_status['🗄️ RDS PostgreSQL'] == '✅ Opérationnel':
    print("   2. ✅ RDS configuré → Passez au notebook 07_setup_rds.ipynb")
else:
    print("   2. ⚠️  Corrigez la configuration RDS avant de continuer")

print("\n" + "="*60)
print("✅ Configuration vérifiée !")
print("="*60)



📋 RÉCAPITULATIF DE LA CONFIGURATION

🔍 État des services AWS :

   📦 S3                 : ✅ Opérationnel
   🗄️ RDS PostgreSQL    : ✅ Opérationnel


🎯 PROCHAINES ÉTAPES :

   1. ✅ S3 configuré → Passez au notebook 06_deploy_s3.ipynb
   2. ✅ RDS configuré → Passez au notebook 07_setup_rds.ipynb

✅ Configuration vérifiée !
